# Neural network hybrid recommendation system on Google Analytics data preprocessing

This notebook demonstrates how to implement a hybrid recommendation system using a neural network to combine content-based and collaborative filtering recommendation models using Google Analytics data. We are going to use the learned user embeddings from [wals.ipynb](../wals.ipynb) and combine that with our previous content-based features from [content_based_using_neural_networks.ipynb](../content_based_using_neural_networks.ipynb)

First we are going to preprocess our data using BigQuery and Cloud Dataflow to be used in our later neural network hybrid recommendation model.

Apache Beam only works in Python 2 at the moment, so we're going to switch to the Python 2 kernel. In the above menu, click the dropdown arrow and select `python2`.

In [12]:
%%bash
source activate py2env
pip uninstall -y google-cloud-dataflow
conda install -y pytz==2018.4
pip install apache-beam[gcp]

Solving environment: ...working... done

# All requested packages already installed.

  Using cached https://files.pythonhosted.org/packages/cf/d5/bd533f864219da7251be3497e622f45f1ac56b1ee2eaa1b601b6b222044f/apache_beam-2.11.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/34/65/a3cd43e1834a748d9650e962ce43083063bf0c778e634d6ae5ca6e62438c/pyarrow-0.11.1-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cc/3e/29f92b7aeda5b078c86d14f550bf85cff809042e3429ace7af6193c3bc9f/typing-3.6.6-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/69/be1402c904506a5ec79ba65c7f0035342d506d5305cfd9ae8bc6f1c8db47/fastavro-0.21.19-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/74/12/065e04ed7bc9c9c45ff08429e0aa74176c4ea63608abb85a4786659889b1/google_apitools-0.5.26-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b7/1b/2b95f2fefddbbece3811

Skipping google-cloud-dataflow as it is not installed.


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base -c defaults conda


google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.8.1 which is incompatible.
googledatastore 7.0.1 has requirement httplib2<0.10,>=0.9.1, but you'll have httplib2 0.11.3 which is incompatible.
Cannot uninstall 'dill'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


Now restart notebook's session kernel!

In [13]:
# Import helpful libraries and setup our project, bucket, and region
import os

PROJECT = 'qwiklabs-gcp-168a579637955d5c' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-168a579637955d5c' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [14]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


<h2> Create ML dataset using Dataflow </h2>
Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

First, let's create our hybrid dataset query that we will use in our Cloud Dataflow pipeline. This will combine some content-based features and the user and item embeddings learned from our WALS Matrix Factorization Collaborative filtering lab that we extracted from our trained WALSMatrixFactorization Estimator and uploaded to BigQuery.

In [15]:
query_hybrid_dataset = """
WITH CTE_site_history AS (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index = 10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index = 7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index = 6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index = 2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index = 4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') AS year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) AS nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index = 10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
),
CTE_training_dataset AS (
SELECT
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) AS next_content_id,
  
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") AS title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") AS author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970, 1, 1), MONTH) AS months_since_epoch
FROM
  CTE_site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL)

SELECT
  CAST(next_content_id AS STRING) AS next_content_id,
  
  CAST(training_dataset.visitor_id AS STRING) AS visitor_id,
  CAST(training_dataset.content_id AS STRING) AS content_id,
  CAST(IFNULL(category, 'None') AS STRING) AS category,
  CONCAT("\\"", REPLACE(TRIM(CAST(IFNULL(title, 'None') AS STRING)), "\\"",""), "\\"") AS title,
  CAST(IFNULL(author, 'None') AS STRING) AS author,
  CAST(months_since_epoch AS STRING) AS months_since_epoch,
  
  IFNULL(user_factors._0, 0.0) AS user_factor_0,
  IFNULL(user_factors._1, 0.0) AS user_factor_1,
  IFNULL(user_factors._2, 0.0) AS user_factor_2,
  IFNULL(user_factors._3, 0.0) AS user_factor_3,
  IFNULL(user_factors._4, 0.0) AS user_factor_4,
  IFNULL(user_factors._5, 0.0) AS user_factor_5,
  IFNULL(user_factors._6, 0.0) AS user_factor_6,
  IFNULL(user_factors._7, 0.0) AS user_factor_7,
  IFNULL(user_factors._8, 0.0) AS user_factor_8,
  IFNULL(user_factors._9, 0.0) AS user_factor_9,
  
  IFNULL(item_factors._0, 0.0) AS item_factor_0,
  IFNULL(item_factors._1, 0.0) AS item_factor_1,
  IFNULL(item_factors._2, 0.0) AS item_factor_2,
  IFNULL(item_factors._3, 0.0) AS item_factor_3,
  IFNULL(item_factors._4, 0.0) AS item_factor_4,
  IFNULL(item_factors._5, 0.0) AS item_factor_5,
  IFNULL(item_factors._6, 0.0) AS item_factor_6,
  IFNULL(item_factors._7, 0.0) AS item_factor_7,
  IFNULL(item_factors._8, 0.0) AS item_factor_8,
  IFNULL(item_factors._9, 0.0) AS item_factor_9,
  
  FARM_FINGERPRINT(CONCAT(CAST(visitor_id AS STRING), CAST(content_id AS STRING))) AS hash_id
FROM CTE_training_dataset AS training_dataset
LEFT JOIN `cloud-training-demos.GA360_test.user_factors` AS user_factors
  ON CAST(training_dataset.visitor_id AS FLOAT64) = CAST(user_factors.user_id AS FLOAT64)
LEFT JOIN `cloud-training-demos.GA360_test.item_factors` AS item_factors
  ON CAST(training_dataset.content_id AS STRING) = CAST(item_factors.item_id AS STRING)
"""

Let's pull a sample of our data into a dataframe to see what it looks like.

In [16]:
import google.datalab.bigquery as bq
df_hybrid_dataset = bq.Query(query_hybrid_dataset + "LIMIT 100").execute().result().to_dataframe()
df_hybrid_dataset.head()

,next_content_id,visitor_id,content_id,category,title,author,months_since_epoch,user_factor_0,user_factor_1,user_factor_2,...,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,item_factor_9,hash_id
0,299925700,1002711106576646390,298972803,News,"""Moslem-Mama: FPÖ-Mann hat selbst iranische Wu...",Matthias Hofer,574,0.000087,0.000168,-0.000745,...,-4.056981e-03,5.865708e-02,-6.528818e-03,-6.191854e-02,1.062048e-01,2.953957e-02,-2.312444e-02,-3.633425e-02,3.527866e-03,-9076280265143853161
1,299972800,1002995626699943875,299836255,News,"""Blümel Kneissl &Co.: Das sind die Fixstarter""",None,574,0.000327,0.000127,-0.000026,...,-5.072439e-05,7.677825e-04,1.595652e-04,3.168983e-04,-4.565390e-04,1.829965e-04,-6.903299e-04,8.621884e-04,1.151190e-04,1905015520082856330
2,299972248,1002995626699943875,299972800,News,"""Strengere Regeln für Schüler Eltern – und auc...",None,574,0.000327,0.000127,-0.000026,...,-6.373322e-28,-2.889791e-29,1.181769e-27,-8.412193e-29,-1.731597e-27,1.196297e-27,-1.205645e-27,2.958057e-28,1.753392e-28,6987272393032641011
3,299972800,1002995626699943875,299972248,News,"""Bildungsexperte Salcher: Lehrer nach Leistung...",Bernhard Gaul,574,0.000327,0.000127,-0.000026,...,-9.316617e-03,1.275338e-01,-7.797281e-03,-5.778661e-02,1.641097e-01,-8.904703e-03,-3.558854e-02,2.324122e-02,3.902660e-02,-7973595518623200848
4,299826767,1002995626699943875,299972800,News,"""Strengere Regeln für Schüler Eltern – und auc...",None,574,0.000327,0.000127,-0.000026,...,-6.373322e-28,-2.889791e-29,1.181769e-27,-8.412193e-29,-1.731597e-27,1.196297e-27,-1.205645e-27,2.958057e-28,1.753392e-28,6987272393032641011


In [17]:
df_hybrid_dataset.describe()

,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,user_factor_9,...,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,item_factor_9,hash_id
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
mean,0.000157,0.001092,-0.000674,-0.001085,-0.001226,0.001176,0.000044,-0.000938,-0.000793,0.000483,...,1.306455e+00,-8.561787e-01,8.277638e-03,-4.769462e-01,7.769683e-01,-1.269643e+00,-2.472956e-01,1.035234e+00,-4.420053e-01,-1.141706e+17
std,0.001768,0.001473,0.002535,0.002132,0.001622,0.001291,0.002107,0.001872,0.001814,0.001486,...,6.357097e+00,4.490344e+00,1.867139e-01,2.031287e+00,3.697427e+00,6.382065e+00,1.002923e+00,5.272180e+00,2.198393e+00,5.454841e+18
min,-0.006352,-0.001665,-0.008577,-0.006265,-0.006195,-0.001782,-0.003510,-0.005085,-0.003952,-0.001535,...,-3.886719e-02,-2.270137e+01,-6.738291e-01,-1.019035e+01,-2.655937e-01,-3.237039e+01,-4.917779e+00,-1.516911e+00,-1.115637e+01,-9.104736e+18
25%,-0.000766,0.000039,-0.000818,-0.001892,-0.002544,0.000041,-0.000956,-0.001754,-0.002587,0.000028,...,-8.069599e-06,-1.169634e-26,-1.822250e-04,-1.752431e-02,-7.421535e-07,-1.637842e-06,-7.357361e-03,-2.270997e-24,-1.015126e-13,-4.707944e+18
50%,0.000301,0.000393,-0.000760,-0.000381,-0.000759,0.001253,-0.000405,-0.000246,-0.000212,0.000213,...,-5.915884e-31,6.912668e-16,0.000000e+00,-9.394841e-14,-1.731597e-27,-1.331895e-24,-8.933754e-08,3.175334e-21,1.973178e-24,-4.615106e+17
75%,0.000878,0.002494,-0.000015,0.000043,-0.000040,0.002175,0.000075,0.000814,0.000239,0.000308,...,9.625795e-05,7.677825e-04,7.853203e-16,-1.507417e-30,1.722620e-03,1.829965e-04,0.000000e+00,8.621884e-04,1.151190e-04,4.202130e+18
max,0.005289,0.005443,0.004528,0.003194,0.002455,0.004147,0.007821,0.001749,0.004731,0.005385,...,3.229185e+01,1.982741e+00,7.682434e-01,2.103930e-02,1.877586e+01,1.030793e+00,1.651876e-02,2.671273e+01,2.130968e-01,9.016824e+18


In [19]:
import apache_beam as beam
import datetime, os

def to_csv(rowdict):
  # Pull columns from BQ and create a line
  import hashlib
  import copy
  CSV_COLUMNS = 'next_content_id,visitor_id,content_id,category,title,author,months_since_epoch'.split(',')
  FACTOR_COLUMNS = ["user_factor_{}".format(i) for i in range(10)] + ["item_factor_{}".format(i) for i in range(10)]
    
  # Write out rows for each input row for each column in rowdict
  data = ','.join(['None' if k not in rowdict else (rowdict[k].encode('utf-8') if rowdict[k] is not None else 'None') for k in CSV_COLUMNS])
  data += ','
  data += ','.join([str(rowdict[k]) if k in rowdict else 'None' for k in FACTOR_COLUMNS])
  yield ('{}'.format(data))
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-features' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/features'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/features/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
    RUNNER = 'DirectRunner'
  else:
    RUNNER = 'DataflowRunner'
  p = beam.Pipeline(RUNNER, options = opts)
  
  query = query_hybrid_dataset

  if in_test_mode:
    query = query + ' LIMIT 100' 

  for step in ['train', 'eval']:
    if step == 'train':
      selquery = 'SELECT * FROM ({}) WHERE MOD(ABS(hash_id), 10) < 9'.format(query)
    else:
      selquery = 'SELECT * FROM ({}) WHERE MOD(ABS(hash_id), 10) = 9'.format(query)

    (p 
     | '{}_read'.format(step) >> beam.io.Read(beam.io.BigQuerySource(query = selquery, use_standard_sql = True))
     | '{}_csv'.format(step) >> beam.FlatMap(to_csv)
     | '{}_out'.format(step) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{}.csv'.format(step))))
    )

  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

ImportError: No module named apache_beam

Let's check our files to make sure everything went as expected

In [20]:
%%bash
rm -rf features
mkdir features

In [21]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/features/*.csv* features/

CommandException: No URLs matched: gs://qwiklabs-gcp-168a579637955d5c/hybrid_recommendation/preproc/features/*.csv*
CommandException: 1 file/object could not be transferred.


In [22]:
!head -3 features/*

head: cannot open 'features/*' for reading: No such file or directory


<h2> Create vocabularies using Dataflow </h2>

Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

Now we'll create our vocabulary files for our categorical features.

In [23]:
query_vocabularies = """
SELECT
  CAST((SELECT MAX(IF(index = index_value, value, NULL)) FROM UNNEST(hits.customDimensions)) AS STRING) AS grouped_by
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,
  UNNEST(hits) AS hits
WHERE
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index = index_value, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  grouped_by
"""

In [24]:
import apache_beam as beam
import datetime, os

def to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out rows for each input row for grouped by column in rowdict
  return '{}'.format(rowdict['grouped_by'].encode('utf-8'))
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-vocab-lists' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/vocabs'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/vocabs/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
      RUNNER = 'DirectRunner'
  else:
      RUNNER = 'DataflowRunner'
      
  p = beam.Pipeline(RUNNER, options = opts)
  
  def vocab_list(index, name):
    query = query_vocabularies.replace("index_value", "{}".format(index))

    (p 
     | '{}_read'.format(name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(name) >> beam.Map(to_txt)
     | '{}_out'.format(name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_vocab.txt'.format(name))))
    )

  # Call vocab_list function for each
  vocab_list(10, 'content_id') # content_id
  vocab_list(7, 'category') # category
  vocab_list(2, 'author') # author
  
  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

ImportError: No module named apache_beam

Also get vocab counts from the length of the vocabularies

In [ ]:
import apache_beam as beam
import datetime, os

def count_to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out count
  return '{}'.format(rowdict['count_number'])
  
def mean_to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out mean
  return '{}'.format(rowdict['mean_value'])
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-vocab-counts' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/vocab_counts'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/vocab_counts/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
      RUNNER = 'DirectRunner'
  else:
      RUNNER = 'DataflowRunner'
      
  p = beam.Pipeline(RUNNER, options = opts)
  
  def vocab_count(index, column_name):
    query = """
SELECT
  COUNT(*) AS count_number
FROM ({})
""".format(query_vocabularies.replace("index_value", "{}".format(index)))

    (p 
     | '{}_read'.format(column_name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(column_name) >> beam.Map(count_to_txt)
     | '{}_out'.format(column_name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_vocab_count.txt'.format(column_name))))
    )
    
  def global_column_mean(column_name):
    query = """
SELECT
  AVG(CAST({1} AS FLOAT64)) AS mean_value
FROM ({0})
""".format(query_hybrid_dataset, column_name)
    
    (p 
     | '{}_read'.format(column_name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(column_name) >> beam.Map(mean_to_txt)
     | '{}_out'.format(column_name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_mean.txt'.format(column_name))))
    )
    
  # Call vocab_count function for each column we want the vocabulary count for
  vocab_count(10, 'content_id') # content_id
  vocab_count(7, 'category') # category
  vocab_count(2, 'author') # author
  
  # Call global_column_mean function for each column we want the mean for
  global_column_mean('months_since_epoch') # months_since_epoch
  
  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

Let's check our files to make sure everything went as expected

In [ ]:
%%bash
rm -rf vocabs
mkdir vocabs

In [ ]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/vocabs/*.txt* vocabs/

In [ ]:
!head -3 vocabs/*

In [ ]:
%%bash
rm -rf vocab_counts
mkdir vocab_counts

In [ ]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/vocab_counts/*.txt* vocab_counts/

In [ ]:
!head -3 vocab_counts/*